In [19]:
from transformers import pipeline
from fastapi import FastAPI, Body, HTTPException
from pydantic import BaseModel

In [5]:
app = FastAPI()

In [22]:
emotion_model = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")
topic_model = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

In [23]:
feedback = "I am absolutely furious about the persistent bug in your app! It's disrupting my workflow, and attempts to get this resolved have been painfully slow. This is unacceptable, and I demand immediate action to fix this issue and prevent further inconvenience."
labels = ['Bug Report', 'Server Outtage', 'Customer Service', 'Billing Error']

In [28]:
topic = topic_model(feedback, labels, multi_label=False)
emotion = emotion_model(feedback)

print(topic['labels'][0]);
print(emotion[0]['label'])

Bug Report
anger


In [1]:
class Feedback(BaseModel):
    content: str
    date: str

class Result:
    content: str
    emotion: str
    tag: str
    date: str

NameError: name 'BaseModel' is not defined

In [29]:
@app.get('/')
def read_root():
    return {'Hello': 'World'}

@app.post('/process/')
async def process_feedback(fd: Feedback):
    
    emotion = emotion_model(fd.content)
    tag = topic_model(fd.content)

    res = {
        'content': fd.content,
        'emotion': await emotion[0]['label'],
        'tag': await tag['labels'][0],
        'date': fd.date
    }    
        
    return {'response': res}
    